### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [56]:
# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [57]:
user_mat = np.random.rand(20, 4) # user matrix filled with random values of shape user x latent 
movie_mat = np.random.rand(4, 4)

In [58]:
movie_mat

array([[ 0.19987384,  0.02653792,  0.1301313 ,  0.18262537],
       [ 0.33701607,  0.14770909,  0.65373121,  0.95927137],
       [ 0.53859931,  0.90243012,  0.73723993,  0.62038214],
       [ 0.93049244,  0.16693103,  0.19121822,  0.2687542 ]])

In [59]:
movie_mat[:, 0]

array([ 0.19987384,  0.33701607,  0.53859931,  0.93049244])

In [60]:
user_mat[0,:]

array([ 0.85405168,  0.27889774,  0.91638575,  0.53735912])

In [61]:
user_mat[0,:]+2*(ratings_mat - np.dot(user_mat, movie_mat))[0,0]*movie_mat[:, 0]*0.0001

array([ 0.85440113,  0.27948696,  0.91732741,  0.53898594])

In [62]:
error = (ratings_mat - np.dot(user_mat, movie_mat))

In [63]:
error[0,1]**2

81.350721755978398

In [64]:
np.square(error).sum()

4730.5258982998303

In [74]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    iteration = 0
    while iteration <= iters:
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for j in range(ratings_mat.shape[1]):
            for i in range(ratings_mat.shape[0]):
                
                # if the rating exists
                    if ~np.isnan(ratings_mat[i, j]):
                        
                    # compute the error as the actual minus the dot product of the user and movie latent features
                        error = (ratings_mat[i,j] - np.dot(user_mat[i,:], movie_mat[:,j]))
                    # Keep track of the total sum of squared errors for the matrix
                        sse = error**2
                        sse_accum = sse_accum + sse
                    # update the values in each matrix in the direction of the gradient
                        user_mat[i,:] = user_mat[i,:] + 2*error*movie_mat[:, j]*learning_rate
                        movie_mat[:,j] = movie_mat[:,j] + 2*(ratings_mat[i,j] - np.dot(user_mat[i,:], movie_mat[:,j]))*user_mat[i,:]*learning_rate
        # print results for iteration
        mse = sse_accum / num_ratings
        print(f"{iteration} | {mse:.1f} ")
        
        iteration = iteration + 1
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [75]:
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.005, 10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
0 | 42.0 
1 | 15.2 
2 | 4.6 
3 | 2.9 
4 | 2.7 
5 | 2.6 
6 | 2.6 
7 | 2.5 
8 | 2.5 
9 | 2.4 
10 | 2.4 


In [76]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.38894039   8.94366573  10.23653663  10.42845493]
 [  8.68667167   7.54404807   9.05605256   8.69047595]
 [  7.6686285    6.78740541   8.63279285   7.68918559]
 [  9.68238259   8.35451303   9.81481428   9.63302888]
 [  8.61964705   7.47442292   8.92795084   8.48822794]
 [  6.44047385   5.87317794   7.92933776   6.52074576]
 [  9.18638708   8.03700696   9.91400153   9.00239289]
 [  8.31785242   7.23755137   8.38255781   8.30447314]
 [  8.47126818   7.3654997    9.23407348   8.21446011]
 [  7.69340717   6.72215581   8.16874617   7.58627074]
 [  8.7941772    7.72670977   9.61305393   8.67967866]
 [  9.50625798   8.40258018   9.97692053   9.71093802]
 [  9.48318991   8.13076261   9.59537383   9.26006826]
 [  6.77622151   5.84515873   6.67803724   6.74128448]
 [  9.744596     8.5189918   10.03432588   9.90401907]
 [  9.64716684   8.48957318  10.58483772   9.474145  ]
 [  8.23234063   7.24006854   8.88422082   8.18788087]
 [  7.56798133   6.32665297   6.36465737   7.76475091]
 [  8.4357

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [77]:
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.005, 250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
0 | 41.5 
1 | 14.3 
2 | 4.3 
3 | 2.9 
4 | 2.7 
5 | 2.7 
6 | 2.7 
7 | 2.7 
8 | 2.6 
9 | 2.6 
10 | 2.6 
11 | 2.6 
12 | 2.6 
13 | 2.5 
14 | 2.5 
15 | 2.5 
16 | 2.4 
17 | 2.4 
18 | 2.4 
19 | 2.3 
20 | 2.3 
21 | 2.2 
22 | 2.1 
23 | 2.1 
24 | 2.0 
25 | 1.9 
26 | 1.9 
27 | 1.8 
28 | 1.7 
29 | 1.6 
30 | 1.6 
31 | 1.5 
32 | 1.4 
33 | 1.3 
34 | 1.2 
35 | 1.2 
36 | 1.1 
37 | 1.0 
38 | 1.0 
39 | 0.9 
40 | 0.8 
41 | 0.8 
42 | 0.7 
43 | 0.6 
44 | 0.6 
45 | 0.5 
46 | 0.5 
47 | 0.4 
48 | 0.4 
49 | 0.3 
50 | 0.3 
51 | 0.3 
52 | 0.2 
53 | 0.2 
54 | 0.2 
55 | 0.1 
56 | 0.1 
57 | 0.1 
58 | 0.1 
59 | 0.1 
60 | 0.1 
61 | 0.1 
62 | 0.0 
63 | 0.0 
64 | 0.0 
65 | 0.0 
66 | 0.0 
67 | 0.0 
68 | 0.0 
69 | 0.0 
70 | 0.0 
71 | 0.0 
72 | 0.0 
73 | 0.0 
74 | 0.0 
75 | 0.0 
76 | 0.0 
77 | 0.0 
78 | 0.0 
79 | 0.0 
80 | 0.0 
81 | 0.0 
82 | 0.0 
83 | 0.0 
84 | 0.0 
85 | 0.0 
86 | 0.0 
87 | 0.0 
88 | 0.0 
89 | 0.0 
90 | 0.0 
91 | 0.0 
92 | 0.0 
93 | 0.0 
94 | 0.0 
9

In [78]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.69522633   9.99999963   9.99999987  10.00000032]
 [ 10.00000034   3.99999968   8.99999989  10.00000009]
 [  8.00000027   9.00000028  10.00000011   4.99999959]
 [  8.99999868   7.99999904   9.99999961  10.00000164]
 [ 10.0000002    5.00000008   9.00000004   8.99999983]
 [  6.00000028   4.          10.00000002   5.99999984]
 [  8.99999936   7.99999822   9.99999936   9.00000198]
 [  9.9999997    5.00000106   9.00000039   7.99999924]
 [  6.9999997    8.00000004  10.00000002   8.00000017]
 [  8.99999961   4.99999954   8.99999984   7.00000066]
 [  8.99999999   8.00000036  10.00000015   7.99999971]
 [  8.99999964  10.00000139  10.00000049   8.999999  ]
 [  9.9999989    8.99999791   9.99999923   8.00000255]
 [  5.00000006   7.99999945   4.99999982   8.00000049]
 [ 10.00000001   7.9999993    9.99999979  10.00000067]
 [  9.00000051   8.99999971   9.99999996  10.        ]
 [  9.00000135   8.00000097   8.00000042   7.99999837]
 [  9.99999999   8.00000048   1.00000018   9.99999959]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [79]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [80]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.005, 250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
0 | 41.3 
1 | 14.4 
2 | 4.3 
3 | 2.9 
4 | 2.7 
5 | 2.7 
6 | 2.7 
7 | 2.6 
8 | 2.6 
9 | 2.6 
10 | 2.5 
11 | 2.5 
12 | 2.5 
13 | 2.4 
14 | 2.4 
15 | 2.3 
16 | 2.3 
17 | 2.2 
18 | 2.1 
19 | 2.1 
20 | 2.0 
21 | 1.9 
22 | 1.8 
23 | 1.8 
24 | 1.7 
25 | 1.7 
26 | 1.6 
27 | 1.5 
28 | 1.5 
29 | 1.4 
30 | 1.4 
31 | 1.4 
32 | 1.3 
33 | 1.3 
34 | 1.2 
35 | 1.2 
36 | 1.2 
37 | 1.1 
38 | 1.1 
39 | 1.1 
40 | 1.0 
41 | 1.0 
42 | 0.9 
43 | 0.9 
44 | 0.9 
45 | 0.8 
46 | 0.8 
47 | 0.7 
48 | 0.7 
49 | 0.7 
50 | 0.6 
51 | 0.6 
52 | 0.6 
53 | 0.6 
54 | 0.5 
55 | 0.5 
56 | 0.5 
57 | 0.5 
58 | 0.5 
59 | 0.4 
60 | 0.4 
61 | 0.4 
62 | 0.4 
63 | 0.4 
64 | 0.4 
65 | 0.3 
66 | 0.3 
67 | 0.3 
68 | 0.3 
69 | 0.3 
70 | 0.2 
71 | 0.2 
72 | 0.2 
73 | 0.2 
74 | 0.2 
75 | 0.2 
76 | 0.2 
77 | 0.1 
78 | 0.1 
79 | 0.1 
80 | 0.1 
81 | 0.1 
82 | 0.1 
83 | 0.1 
84 | 0.1 
85 | 0.1 
86 | 0.1 
87 | 0.0 
88 | 0.0 
89 | 0.0 
90 | 0.0 
91 | 0.0 
92 | 0.0 
93 | 0.0 
94 | 0.0 
9

In [81]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.18087226108087:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [82]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, 4, 0.005, 20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
0 | 23.4 
1 | 10.8 
2 | 7.5 
3 | 5.8 
4 | 4.7 
5 | 3.9 
6 | 3.4 
7 | 2.9 
8 | 2.6 
9 | 2.3 
10 | 2.1 
11 | 1.9 
12 | 1.7 
13 | 1.6 
14 | 1.5 
15 | 1.4 
16 | 1.3 
17 | 1.2 
18 | 1.1 
19 | 1.1 
20 | 1.0 


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [83]:
first_1000_users

matrix([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [ ]:
# Replace each of the comments below with the correct values
num_ratings = # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")